# Arrays and views

Julia has excellent functionality for manipulating arrays and for linear algebra. We will have a quick look at this subject, which is much more complicated than you might suspect; see e.g. the talk on "Taking vector transposes seriously".

Let's define a 2x2 array (matrix):

In [2]:
M = [1 2 3; 4 5 6; 7 8 9]  # a 3x3 matrix

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

In [3]:
typeof(M)

Array{Int64,2}

We can extract part of the matrix using indexing notation:

In [5]:
part = M[2:3, 1:2]

2×2 Array{Int64,2}:
 4  5
 7  8

What happens if we modify `part`?

In [7]:
part[1, 1]

4

In [8]:
part[1, 1] = 10

10

In [9]:
part

2×2 Array{Int64,2}:
 10  5
  7  8

In [10]:
M

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

We see that `M` has *not* been modified: `part` was a **copy** of that part of `M`.

## Views

We often do *not* want a copy, but rather just a reference to the same data, which is called a `view`: 

In [17]:
V = view(M, 2:3, 1:2)

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 4  5
 7  8

In [18]:
typeof(V)

SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}

Although this type looks complicated, it just contains the necessary information for the object to manipulate correctly the underlying data.

If we modify `V`, then `M` also gets modified, since it is the same data:

In [21]:
V[1, 1]

4

In [22]:
V[1, 1] = 100

100

In [23]:
V

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 100  5
   7  8

In [24]:
M

3×3 Array{Int64,2}:
   1  2  3
 100  5  6
   7  8  9

We can also write

In [19]:
@view M[2:3, 1:2]

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 4  5
 7  8

for ease of use.

## In-place and vectorized operations: "`.`" ("pointwise")

Suppose we have two matrices and wish to add one to the other:

In [41]:
A = rand(1000, 1000)
B = rand(1000, 1000);

Coming from other languages, we might expect to write `A += B`, and indeed this works:

In [42]:
A += B

1000×1000 Array{Float64,2}:
 0.968928  1.32343   1.24209   0.872131  …  0.720564  0.388369  1.14116 
 0.204676  0.421446  1.54268   0.566509     0.976438  1.10873   1.57093 
 0.581773  0.199507  0.815626  1.15475      0.972806  0.828963  0.808414
 1.13841   1.56143   0.746309  1.95236      0.115347  1.24063   0.72575 
 1.19113   1.07349   0.553199  1.16877      1.72498   1.55993   0.822072
 1.0875    1.462     1.4386    0.608865  …  0.982909  1.60436   0.439304
 1.01382   0.684245  1.16893   0.945915     0.754567  1.52645   1.57565 
 0.949518  0.58178   1.10361   1.02915      0.713869  0.713696  0.940816
 0.998316  1.24432   0.311364  0.975239     0.542564  0.821899  0.749195
 1.51759   1.41818   0.788877  0.463206     0.349169  1.31018   1.37438 
 1.29745   0.763401  1.23604   1.598     …  0.88559   0.860827  0.438045
 1.11474   1.13203   1.12227   0.400706     1.64049   0.387016  1.64967 
 1.66822   0.693214  1.29294   0.393161     1.38031   0.559444  1.33583 
 ⋮                     

This is just "syntactic sugar" (i.e. a cute way of writing) `A = A + B`.

However, it turns out that this does not do what you think it does, namely "in-place addition", in which each element of `A` is updated in place. Rather, it allocates a new temporary object for the result of `A + B`. We can see this:

In [43]:
using BenchmarkTools

@btime $A += $B;

  1.757 ms (2 allocations: 7.63 MiB)


1000×1000 Array{Float64,2}:
 1.34548   1.78995   1.49163   1.4206    …  1.0518    0.552815  2.00633 
 0.213293  0.798752  2.27662   0.687972     1.46436   1.7575    2.26951 
 0.585912  0.28548   1.01223   2.09057      0.97898   0.99713   1.55843 
 1.90392   2.50291   0.861714  2.9059       0.119474  1.48763   0.818001
 1.81095   1.69645   1.09744   2.13924      2.46413   2.50837   1.14472 
 1.25784   2.05733   1.94274   0.62532   …  1.24706   2.27125   0.576669
 1.984     1.18045   1.87166   1.32362      1.18667   2.28369   2.28583 
 1.21146   0.915668  1.61834   1.73474      0.965952  1.26704   1.06328 
 1.15399   2.05755   0.341304  1.56319      1.04554   1.54696   1.48006 
 2.07014   1.99058   1.45165   0.877347     0.3983    1.79719   2.29727 
 2.07513   1.02964   2.02191   2.23425   …  0.976161  1.48534   0.871693
 1.55057   1.41962   1.42887   0.546927     2.60417   0.650841  2.34986 
 2.35675   1.24106   2.02715   0.578252     1.84711   0.721913  2.00956 
 ⋮                     

Note the large amount of allocation here (1,000,000 $\times$ 8 bytes)

The in-place behaviour can be obtained using **pointwise operators**:

In [44]:
A .= A .+ B

1000×1000 Array{Float64,2}:
 1.34548   1.78995   1.49163   1.4206    …  1.0518    0.552815  2.00633 
 0.213293  0.798752  2.27662   0.687972     1.46436   1.7575    2.26951 
 0.585912  0.28548   1.01223   2.09057      0.97898   0.99713   1.55843 
 1.90392   2.50291   0.861714  2.9059       0.119474  1.48763   0.818001
 1.81095   1.69645   1.09744   2.13924      2.46413   2.50837   1.14472 
 1.25784   2.05733   1.94274   0.62532   …  1.24706   2.27125   0.576669
 1.984     1.18045   1.87166   1.32362      1.18667   2.28369   2.28583 
 1.21146   0.915668  1.61834   1.73474      0.965952  1.26704   1.06328 
 1.15399   2.05755   0.341304  1.56319      1.04554   1.54696   1.48006 
 2.07014   1.99058   1.45165   0.877347     0.3983    1.79719   2.29727 
 2.07513   1.02964   2.02191   2.23425   …  0.976161  1.48534   0.871693
 1.55057   1.41962   1.42887   0.546927     2.60417   0.650841  2.34986 
 2.35675   1.24106   2.02715   0.578252     1.84711   0.721913  2.00956 
 ⋮                     

In [45]:
@btime A .= A .+ B

  725.688 μs (4 allocations: 128 bytes)


1000×1000 Array{Float64,2}:
  4304.94     5333.65   2853.48    …   3786.79     1880.01    9890.04  
    98.6948   4313.03   8390.52        5577.88     7416.65    7986.29  
    47.8988    982.864  2247.98          71.5438   1922.97    8573.52  
  8750.85    10762.6    1319.82          47.2898   2824.45    1055.16  
  7085.71     7121.46   6221.25        8450.19    10842.3     3688.7   
  1948.09     6806.05   5763.76    …   3020.24     7624.17    1570.52  
 11090.3      5672.31   8033.33        4939.69     8656.73    8119.04  
  2994.99     3816.92   5884.44        2882.02     6325.49    1400.66  
  1780.37     9296.46    342.526       5749.6      8288.25    8354.55  
  6317.08     6544.0    7576.33         561.921    5567.8    10550.0   
  8890.2      3043.9    8983.72    …   1036.11     7139.07    4957.04  
  4982.6      3288.36   3505.58       11016.4      3015.9     8004.85  
  7871.65     6262.57   8393.34        5336.95     1857.58    7702.03  
     ⋮                             ⋱

Furthermore, we can chain such operations together with no creation of temporaries:

In [46]:
C = rand(1000, 1000)

@btime A .+= B + C  # allocates

  2.626 ms (6 allocations: 7.63 MiB)


1000×1000 Array{Float64,2}:
  6871.03    7508.43   6151.07  10138.4    …   5878.45    4342.9   12620.3 
  2263.17    5360.1   11577.1    2205.83       6958.22    9295.74  11218.5 
   483.878   1856.49   3755.18  14813.2         115.145   2609.76  11516.7 
 12125.8    14842.5    3490.5   15315.2        1769.31    5973.3    2211.09
  9887.45   10989.5    9158.5   15452.5       12181.2    14525.2    6659.89
  4005.47   10119.7    7320.63   2867.35   …   4612.68   10629.1    2402.8 
 15877.0     7978.97  11259.8    5609.87       7807.92   12885.3   10213.0 
  3712.99    5512.19   9497.4   11834.9        5358.9     8356.29   2570.53
  4808.12   13984.4    2802.51  10297.9        7302.3    12307.5   11400.4 
  8550.36    9887.78  11272.6    7755.13       2589.72    7357.38  13315.8 
 11759.0     5040.06  13658.2    9208.91   …   1415.75    9477.93   6824.76
  6539.19    5129.52   4796.1    2844.78      15438.9     5459.98  10899.8 
 11661.3     8025.83  12628.0    2743.05       7688.21    35

In [47]:
@btime A .+= B .+ C  # does not allocate

  1.128 ms (4 allocations: 160 bytes)


1000×1000 Array{Float64,2}:
 14146.5   13674.5   15500.6   21106.7   …  11808.8    11325.8   20361.1 
  8400.02   8328.81  20612.0    4522.03     10871.8    14623.4   20382.8 
  1719.99   4333.43   8028.48  26482.2        238.767   4556.97  19861.2 
 21694.6   26409.9    9644.93  27830.7       6651.68   14901.1    5488.46
 17831.1   21956.4   17486.4   27811.0      22759.5    24967.1   15084.0 
  9838.66  19514.7   11734.7   10462.1   …   9127.62   19148.9    4762.53
 29448.8   14519.0   20407.8    9272.27     15940.1    24874.6   16150.0 
  5748.72  10318.7   19741.1   22521.2      12381.5    14114.1    5887.41
 13392.6   27275.9    9777.21  20438.6      11704.6    23703.1   20036.2 
 14882.3   19368.3   21752.3   16320.5       8339.06   12431.3   21157.8 
 19892.8   10699.7   26911.5   14695.3   …   2492.12   16109.2   12120.2 
 10952.5   10349.7    8455.05   6170.75     27977.8    12389.6   19107.6 
 22405.8   13025.1   24634.4    4520.95     14354.6     8298.81  24256.0 
     ⋮    

See [this blog post by Steven Johnson](https://julialang.org/blog/2017/01/moredots) for more details.

## Efficient small matrices and vectors

For small matrices and vectors, the generic vector and matrix code is too slow, since the type does not contain the information on the number of elements contained in the array, so that generic loops are used.

The `StaticArrays.jl` package fixes this problem by unrolling operations for small arrays.

In [48]:
# Pkg.add("StaticArrays")

using StaticArrays, BenchmarkTools

In [25]:
function bench()
    x = SVector(1, 2)
    y = [1, 2]
    
    @btime $x + $x
    @btime $y + $y
end

bench (generic function with 1 method)

In [26]:
bench()

  1.380 ns (0 allocations: 0 bytes)
  38.239 ns (1 allocation: 96 bytes)


2-element Array{Int64,1}:
 2
 4

In [28]:
x = SVector(1, 2)
@code_lowered x + x

CodeInfo(:(begin 
        nothing
        nothing
        return (StaticArrays.map)(StaticArrays.+, a, b)
    end))

In [29]:
@code_typed x + x

CodeInfo(:(begin 
        SSAValue(4) = a
        SSAValue(5) = b
        $(Expr(:inbounds, false))
        # meta: location /Users/dpsanders/.julia/v0.6/StaticArrays/src/mapreduce.jl map 10
        SSAValue(2) = SSAValue(4)
        SSAValue(3) = SSAValue(5)
        # meta: location /Users/dpsanders/.julia/v0.6/StaticArrays/src/mapreduce.jl _map 14
        # meta: location /Users/dpsanders/.julia/v0.6/StaticArrays/src/mapreduce.jl # line 23:
        $(Expr(:inbounds, true))
        #temp# = $(Expr(:new, SVector{2,Int64}, :((StaticArrays.tuple)((Base.add_int)((Base.getfield)((Core.getfield)(SSAValue(2), :data)::Tuple{Int64,Int64}, 1)::Int64, (Base.getfield)((Core.getfield)(SSAValue(3), :data)::Tuple{Int64,Int64}, 1)::Int64)::Int64, (Base.add_int)((Base.getfield)((Core.getfield)(SSAValue(2), :data)::Tuple{Int64,Int64}, 2)::Int64, (Base.getfield)((Core.getfield)(SSAValue(3), :data)::Tuple{Int64,Int64}, 2)::Int64)::Int64)::Tuple{Int64,Int64})))
        goto 15
        # meta: pop location


In [30]:
@code_llvm x + x


define void @"julia_+_61704"(%SArray* noalias nocapture sret, %SArray* nocapture readonly dereferenceable(16), %SArray* nocapture readonly dereferenceable(16)) #0 !dbg !5 {
top:
  %3 = getelementptr inbounds %SArray, %SArray* %1, i64 0, i32 0, i64 0
  %4 = getelementptr inbounds %SArray, %SArray* %2, i64 0, i32 0, i64 0
  %5 = load i64, i64* %3, align 8
  %6 = load i64, i64* %4, align 8
  %7 = add i64 %6, %5
  %8 = getelementptr inbounds %SArray, %SArray* %1, i64 0, i32 0, i64 1
  %9 = getelementptr inbounds %SArray, %SArray* %2, i64 0, i32 0, i64 1
  %10 = load i64, i64* %8, align 8
  %11 = load i64, i64* %9, align 8
  %12 = add i64 %11, %10
  %"#temp#.sroa.0.sroa.0.0.#temp#.sroa.0.0..sroa_cast1.sroa_idx" = getelementptr inbounds %SArray, %SArray* %0, i64 0, i32 0, i64 0
  store i64 %7, i64* %"#temp#.sroa.0.sroa.0.0.#temp#.sroa.0.0..sroa_cast1.sroa_idx", align 8
  %"#temp#.sroa.0.sroa.2.0.#temp#.sroa.0.0..sroa_cast1.sroa_idx7" = getelementptr inbounds %SArray, %SArray* %0, i64 0, i32

In [31]:
@code_native x + x

	.section	__TEXT,__text,regular,pure_instructions
Filename: linalg.jl
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 23
	movq	(%rdx), %rax
	movq	8(%rdx), %rcx
	addq	(%rsi), %rax
	addq	8(%rsi), %rcx
Source line: 10
	movq	%rax, (%rdi)
	movq	%rcx, 8(%rdi)
	movq	%rdi, %rax
	popq	%rbp
	retq
	nop


In [32]:
y = [1, 2]
@code_native y + y

	.section	__TEXT,__text,regular,pure_instructions
Filename: arraymath.jl
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rbx
	subq	$64, %rsp
	movq	%rsi, %r12
	movq	%rdi, %r15
	movabsq	$jl_get_ptls_states_fast, %rax
	callq	*%rax
	movq	%rax, %r14
	movq	$0, -40(%rbp)
	movq	$0, -48(%rbp)
	movq	$4, -64(%rbp)
	movq	(%r14), %rax
	movq	%rax, -56(%rbp)
	leaq	-64(%rbp), %rax
	movq	%rax, (%r14)
Source line: 64
	movq	24(%r15), %rax
Source line: 64
	movq	24(%r12), %rcx
	xorl	%ebx, %ebx
Source line: 37
	testq	%rax, %rax
	cmovsq	%rbx, %rax
	movq	%rax, -72(%rbp)
	testq	%rcx, %rcx
	cmovsq	%rbx, %rcx
	movq	%rcx, -80(%rbp)
	movabsq	$promote_shape, %rax
	leaq	-72(%rbp), %rdi
	leaq	-80(%rbp), %rsi
	callq	*%rax
Source line: 64
	movq	24(%r15), %rax
Source line: 64
	movq	24(%r12), %rcx
Source line: 63
	testq	%rax, %rax
	cmovsq	%rbx, %rax
	movq	%rax, -88(%rbp)
	testq	%rcx, %rcx
	cmovsq	%rbx, %rcx
	movq	%rcx, -96(%rbp)
	movabsq	$_bcs1, %rax
	leaq	-88(%rbp), %rdi
	leaq	-96(%rbp), %rsi
	c